<div align="center"><img src="../images/LKYCIC_Header.jpg"></div>

**Table of contents**<a id='toc0_'></a>    
- [2-02: Raster Data Analysis](#toc1_)    
  - [Remote Sensing Image Processing](#toc1_1_)    
    - [Read GeoTIFF files](#toc1_1_1_)    
      - [Multi-band TIFF](#toc1_1_1_1_)    
  - [Data](#toc1_2_)    
  - [Clip raster from Polygon](#toc1_3_)    
  - [Calculation](#toc1_4_)    
    - [Plot the raster](#toc1_4_1_)    
    - [Filter Values](#toc1_4_2_)    
    - [Remote Sensing Analysis and Urban Research](#toc1_4_3_)    
  - [Next Section](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[2-02: Raster Data Analysis](#toc0_)

Raster data consists of cells of a fixed size and is widely used in **remote sensing**. In this section, we will calculate **Normalized Difference Vegetation Index (NDVI)**, plot it and filter the NDVI index value, showcasing a practical application of raster analysis.

## <a id='toc1_1_'></a>[Remote Sensing Image Processing](#toc0_)

<div align="center">
    <img src="../images/raster_concept.png" width="50%">
    <br><b>Value of a pixel in a raster</b>
    <br><u>Source: National Ecological Observatory Network (NEON)</u>
</div>

The most common raster data format is the **GeoTIFF**. This format is a standard for georeferencing information and is widely used in the remote sensing community. In this notebook, we will learn how to read and write GeoTIFF files using Python.

### <a id='toc1_1_1_'></a>[Read GeoTIFF files](#toc0_)

#### <a id='toc1_1_1_1_'></a>[Multi-band TIFF](#toc0_)

A raster can contain one or more bands. 

One type of multi-band raster dataset that is familiar to many of us is a coloured image:

<div align="center">
    <img src="../images/RGBSTack_1.jpg" width="50%">
    <br><b>RGB bands (Red, Green, Blue)</b>
    <br><u>Source: National Ecological Observatory Network (NEON)</u>
</div>

## <a id='toc1_2_'></a>[Data](#toc0_)

We will be using [landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1) satellite images. Average values from Jan 01, 2024 to Jan 30, 2024 in the area of Ho Chi Minh City.

How to download remote sensing data:

**1. Landsat (NASA/USGS)**
Description:
- Long-term satellite imagery with moderate spatial resolution (30 m for most bands).
- Available from Landsat 1 to the current Landsat 9.

Applications:
- Land cover classification, vegetation monitoring, urban studies.

How to Download:

**Google Earth Engine (GEE)**

Cloud-based geospatial analysis platform with access to multiple datasets, including Landsat, Sentinel, MODIS, and more.

How to Access:

- **[GEE Directly](https://earthengine.google.com/)**:

  1. Apply for free access.

  2. Write scripts in JavaScript to analyse and export data.

- **[GEEMap](https://geemap.org/)**:

  1. A Python package that interfaces with GEE. 
  
  2. geemap simplifies the process of accessing and visualising geospatial datasets directly from GEE.

Rasterio will be used to read the TIFF file.

Install the rasterio package:

In [ ]:
# uncomment to install rasterio
#!pip install rasterio

In [ ]:
import rasterio

Firstly, we can print the meta data:

In [ ]:
# Open the GeoTIFF file
with rasterio.open('../data/raw/part_ii/landsat8_hcmc/lansat_hcmc_jan2024.tif') as dataset:
    
    # Read the dataset's metadata
    print(dataset.meta)

    # Get the number of bands
    num_bands = dataset.count

    # Get the ID of the bands
    band_names = dataset.indexes

    # Get the crs
    crs = dataset.crs
    epsg = crs.to_epsg()

# Print the number of bands
print(f'Number of bands: {num_bands}')

# Print the band names
print(f'Band names: {band_names}')

# Print the crs
print(f'CRS: {crs}')
print(f'EPSG: {epsg}')

## <a id='toc1_3_'></a>[Clip raster from Polygon](#toc0_)

In [ ]:
import geopandas as gpd

hcmc_boundary = gpd.read_file('../data/raw/part_iii/HCMC_Boundary.geojson')

hcmc_boundary.plot()

In [ ]:
# Check the CRS of the boundary equal to the CRS of the raster
hcmc_boundary.crs == crs

In [ ]:
# clip the NDVI raster with the boundary of Ho Chi Minh City
from rasterio.mask import mask

with rasterio.open('../data/raw/part_ii/landsat8_hcmc/lansat_hcmc_jan2024.tif') as src:
    lansat_clipped, lansat_transform = mask(src, hcmc_boundary.geometry, crop=True)
    lansat_meta = src.meta

# Print the shape of the clipped NDVI raster
print(lansat_clipped.shape)

There is only two bands within the GeoTIFF.

To calculate the NDVI map for the city of Ho Chi Minh City, Vietnam.

It is calculated using the reflectance values of the red (R) and near-infrared (NIR) bands of the electromagnetic spectrum. The formula for NDVI is:

$$
\text{NDVI} = \frac{(\text{NIR} - \text{Red})}{(\text{NIR} + \text{Red})}
$$

Where:

- NIR is the reflectance in the near-infrared band.

- Red is the reflectance in the red band.

The GeoTiff is preprocessed, where:

- Band 1 is the NIR band;

- Band 2 is the red band.

In [ ]:
NIR_band = lansat_clipped[0]

Red_band = lansat_clipped[1]

In [ ]:
type(NIR_band)

When you read a raster file using libraries like `rasterio`, it returns the data as a `NumPy ndarray` because the primary content of a raster file is a grid of pixel values, which can be naturally represented as `a multi-dimensional array`. This array contains the actual data values (e.g., elevation, temperature, reflectance) for each pixel in the raster.

However, the geographical information (such as the coordinate reference system, geotransform, and extent) is not lost. This information is stored in `the metadata` of the raster file and can be accessed separately.

## <a id='toc1_4_'></a>[Calculation](#toc0_)

$$
\text{NDVI} = \frac{(\text{NIR} - \text{Red})}{(\text{NIR} + \text{Red})}
$$

In [ ]:
ndvi_hcmc = (NIR_band - Red_band) / (NIR_band + Red_band)

The NDVI values range from -1 to 1:

- Values close to 1 indicate healthy, dense vegetation.

- Values close to 0 indicate sparse or no vegetation.

- Negative values indicate non-vegetated surfaces such as water, snow, or bare soil.

In [ ]:
ndvi_hcmc.shape

### <a id='toc1_4_1_'></a>[Plot the raster](#toc0_)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 10)) # Set the size of the plot
plt.imshow(ndvi_hcmc, cmap='Greens') # Display the NDVI using a grayscale colormap
plt.colorbar() # Display the colorbar
plt.title('NDVI Ho Chi Minh City 2024') # Set the title of the plot
plt.show()

In [ ]:
import numpy as np
from matplotlib.colors import ListedColormap

### <a id='toc1_4_2_'></a>[Filter Values](#toc0_)

Let's take a value of "0.3" as threshold value of vegetation.

Which means:

VoP >= 0.3: Vegetation cover

VoP < 0.3: Other Surface Material

*Note: 0.3 is selected for demonstration, not scientifically derived*

Let's extract the values of pixels with NDVI values greater than 0.3 as a new array.

In [ ]:
# Create a new array with the NDVI values greater than 0.3
ndvi_hcmc_high = np.where(ndvi_hcmc > 0.3, ndvi_hcmc, np.nan)

In [ ]:
plt.figure(figsize=(10, 10)) # Set the size of the plot
plt.imshow(ndvi_hcmc_high, cmap='Greens') # Display the NDVI using a green colormap
plt.colorbar() # Display the colorbar
plt.title('Vegetation Cover HCMC 2024') # Set the title of the plot
plt.show()

In [ ]:
# save as tif
with rasterio.open('../data/processed/part_ii/processed_ndvi/ndvi_hcmc_2024_high.tif', # Name of the output file
                   'w', # Write the file
                   driver='GTiff', # Specify the driver to be used
                   height=ndvi_hcmc_high.shape[0], # Dimensions of the image 
                   width=ndvi_hcmc_high.shape[1],  # Dimensions of the image
                   count=1,  # Number of bands
                   dtype=ndvi_hcmc_high.dtype, # Use the same datatype as the input file
                   crs=crs, # Use the same crs as the input file
                   transform=dataset.transform # Use the same transformation as the input file
                   ) as dst:
    dst.write(ndvi_hcmc_high, 1) # Write the new band to the file


<div align="center">
    <img src="../images/ndvi_vis.gif" width="50%">
    <br><b>Vegetation identification through NDVI in HCMC.</b>
</div>

### <a id='toc1_4_3_'></a>[Remote Sensing Analysis and Urban Research](#toc0_)

The advantages:  

- **Large-Scale Coverage**: Remote sensing provides data over vast areas

- **High Temporal Resolution**: Frequent data collection allows for monitoring changes over time, such as urban expansion, deforestation, or disaster impact assessment.  

- **Non-Invasive Data Collection**: Remote sensing is a passive, non-invasive method, making it ideal for studying inaccessible, sensitive, or hazardous areas without direct fieldwork.  

## <a id='toc1_5_'></a>[Next Section](#toc0_)

Go to [2-03: Network Analysis](./2-03_network.ipynb)